In [1]:
import os
import gen_utils as gu
import torch
import numpy as np
from dataset.common import normal_redirect
# import open3d as o3d
# import pyvista as pv

import pymeshlab

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


palet : 치아 레이블 별 색깔
(치아 레이블 별로 색필 => 색칠된 상태에서 샘플링 => 샘플링된 포인트의 색깔을 기준으로 다시 레이블링)

In [2]:
# palet = np.array([
#     [255,153,153],

#     [153,76,0],
#     [153,153,0],
#     [76,153,0],
#     [0,153,153],
#     [0,0,153],
#     [153,0,153],
#     [153,0,76],
#     [64,64,64],

#     [255,128,0],
#     [153,153,0],
#     [76,153,0],
#     [0,153,153],
#     [0,0,153],
#     [153,0,153],
#     [153,0,76],
#     [64,64,64],
#     ])/255
# palet[9:] *= 0.4
palet = np.array([
    [255,153,153],

    [153,76,0],
    [153,153,0],
    [76,153,0],
    [0,153,153],
    [0,0,153],
    [153,0,153],
    [153,0,76],
    [64,64,64],

    [20, 10, 0],
    [10, 10, 0],
    [10, 20, 0],
    [0, 10, 20],
    [0, 0, 20],
    [10, 0, 10],
    [10, 0, 0],
    [10, 10, 10],
    ])/255

In [3]:
split_base_name_ls = []
stl_path_ls = []
json_path_map = {}

source_obj_path = "../datasets/3D_scans_per_patient_obj_files"
source_json_path = "../datasets/ground-truth_labels_instances"


## Train dataset & Test dataset 분리
for split_with_txt_path in ['base_name_train_fold.txt', 'base_name_test_fold.txt']:
    f = open(split_with_txt_path, 'r')
    while True:
        line = f.readline()
        if not line: break
        split_base_name_ls.append(line.strip())
    f.close()

for case in split_base_name_ls:
    stl_path_ls.append(os.path.join(source_obj_path, case, case+'_upper.obj'))
    stl_path_ls.append(os.path.join(source_obj_path, case, case+'_lower.obj'))
    
for case in split_base_name_ls:
    json_path_map[case+'_upper'] = os.path.join(source_json_path, case, case+'_upper.json')
    json_path_map[case+'_lower'] = os.path.join(source_json_path, case, case+'_lower.json')

# fps

In [5]:
save_path = './preprocessed_data/'

for stl_path in stl_path_ls:
    base_name = os.path.basename(stl_path).split(".")[0]
    print("base_name : ", base_name)
    loaded_json = gu.load_json(json_path_map[base_name])
    labels = torch.tensor(loaded_json['labels']).reshape(-1)
    if loaded_json['jaw'] == 'lower':
        labels -= 20
    labels[labels//10==1] %= 10
    labels[labels//10==2] = (labels[labels//10==2]%10) + 8
    labels[labels<0] = 0
    
    
    vertices, org_mesh = gu.read_txt_obj_ls(stl_path, ret_mesh=True, use_tri_mesh=False)
    
    '''Previous Sampling - FPS'''
    labeled_vertices = np.concatenate([vertices, np.expand_dims(labels, -1)], 1)
    
    if labeled_vertices.shape[0]>24000:
        labeled_vertices = gu.resample_pcd([labeled_vertices], 24000, "fps")[0]
    ''''''
    
    
    np.save(os.path.join(save_path, base_name+'.npy'), labeled_vertices)

base_name :  4OPR2WF4_upper

vertices :  (143741, 6)
labels :  torch.Size([143741])

base_name :  4OPR2WF4_lower

vertices :  (92981, 6)
labels :  torch.Size([92981])

base_name :  BIIEY91S_upper

vertices :  (191254, 6)
labels :  torch.Size([191254])

base_name :  BIIEY91S_lower

vertices :  (112279, 6)
labels :  torch.Size([112279])

base_name :  85ARPPFY_upper

vertices :  (147382, 6)
labels :  torch.Size([147382])

base_name :  85ARPPFY_lower

vertices :  (95615, 6)
labels :  torch.Size([95615])

base_name :  01HXMTNF_upper

vertices :  (126994, 6)
labels :  torch.Size([126994])

base_name :  01HXMTNF_lower

vertices :  (89173, 6)
labels :  torch.Size([89173])

base_name :  U1HFBZ1F_upper

vertices :  (179007, 6)
labels :  torch.Size([179007])

base_name :  U1HFBZ1F_lower

vertices :  (117425, 6)
labels :  torch.Size([117425])

base_name :  L7Z4TSKP_upper

vertices :  (212473, 6)
labels :  torch.Size([212473])

base_name :  L7Z4TSKP_lower

vertices :  (153372, 6)
labels :  torch.Si

# sample_point_poisson_disk

In [4]:
for stl_path in stl_path_ls:
    base_name = os.path.basename(stl_path).split(".")[0]
    print("base_name : ", base_name)
    loaded_json = gu.load_json(json_path_map[base_name])
    labels = torch.tensor(loaded_json['labels']).reshape(-1,1)
    if loaded_json['jaw'] == 'lower':
        labels -= 20
    labels[labels//10==1] %= 10
    labels[labels//10==2] = (labels[labels//10==2]%10) + 8
    labels[labels<0] = 0
    
    
    vertices, org_mesh = gu.read_txt_obj_ls(stl_path, ret_mesh=True, use_tri_mesh=False)
    colored_mesh = gu.get_colored_mesh(org_mesh, np.array(labels.reshape(-1)))

    pcd = colored_mesh.sample_points_poisson_disk(24000)
    vertices = np.array(pcd.points)
    normals = np.array(pcd.normals)
    color_ind = np.array(pcd.colors)
    sampled_label = np.zeros(24000)
    for i, p in enumerate(palet):
        sampled_label[((np.isclose(color_ind, p)).sum(-1)//3).astype(np.bool_)] = i
        
    labeled_vertices = np.concatenate([vertices, normals, np.expand_dims(sampled_label, axis=-1).astype(np.int64)], 1)
    
    
    labeled_vertices[:, :3] -= np.mean(labeled_vertices[:, :3], axis=0)
    # labeled_vertices[:, :3] = labeled_vertices[:, :3] / np.max(np.abs(labeled_vertices[:, :3]))
    # labeled_vertices[:, :3] = ((labeled_vertices[:, :3]-self.Y_AXIS_MIN)/(self.Y_AXIS_MAX - self.Y_AXIS_MIN))*2-1
    
    src_normals = normal_redirect(labeled_vertices[:, :3], labeled_vertices[:, 3:6], view_point=np.array([0., 0., 0.]))
    labeled_vertices[:, 3:6] = src_normals
    
    # np.save(os.path.join(save_path, base_name+'.npy'), labeled_vertices)
    
    
    # # o3d.visualization.draw_geometries([pcd])
    
    # cloud = pv.PolyData(src_pcd[:,:3])
    # mesh = cloud.delaunay_2d()
    
    # src_pcd = np.concatenate([np.array(mesh.points), np.array(mesh.point_normals)], axis=-1)

    # org_mesh.vertices = o3d.utility.Vector3dVector(np.array(mesh.points))
    # org_mesh.triangles = o3d.utility.Vector3iVector(np.array(mesh.regular_faces))
    # org_mesh.compute_vertex_normals()
    
    # cls_gt_colored_mesh = gu.get_colored_mesh(org_mesh, torch.tensor(sampled_label+1, device='cuda').detach().cpu().numpy())
    # gu.print_3d(cls_gt_colored_mesh)

base_name :  4OPR2WF4_upper
base_name :  4OPR2WF4_lower
base_name :  BIIEY91S_upper
base_name :  BIIEY91S_lower
base_name :  85ARPPFY_upper
base_name :  85ARPPFY_lower
base_name :  01HXMTNF_upper
base_name :  01HXMTNF_lower
base_name :  U1HFBZ1F_upper
base_name :  U1HFBZ1F_lower
base_name :  L7Z4TSKP_upper
base_name :  L7Z4TSKP_lower
base_name :  01KRCSKD_upper
base_name :  01KRCSKD_lower
base_name :  01J24RCK_upper
base_name :  01J24RCK_lower
base_name :  01F6WV5D_upper
base_name :  01F6WV5D_lower
base_name :  014ZWMZ8_upper
base_name :  014ZWMZ8_lower
base_name :  5J9O0LG3_upper
base_name :  5J9O0LG3_lower
base_name :  01JHAGK0_upper
base_name :  01JHAGK0_lower
base_name :  H83RDQT1_upper
base_name :  H83RDQT1_lower
base_name :  019T94KX_upper
base_name :  019T94KX_lower
base_name :  01A91JH6_upper
base_name :  01A91JH6_lower
base_name :  DXNHHADD_upper
base_name :  DXNHHADD_lower
base_name :  01FUZ18R_upper
base_name :  01FUZ18R_lower
base_name :  017U3R3T_upper
base_name :  017U3R3

# **generate_simplified_point_cloud**

In [16]:
save_path = './preprocessed_data_simplification/'

for stl_path in stl_path_ls:
    base_name = os.path.basename(stl_path).split(".")[0]
    print("base_name : ", base_name)
    loaded_json = gu.load_json(json_path_map[base_name])
    labels = torch.tensor(loaded_json['labels']).reshape(-1)
    
    
    ### 치아 번호들을 0(잇몸), 1~8(#11~#18, #31~#38), 9~16(#21~#28, #41~48) 로 변경해줌
    if loaded_json['jaw'] == 'lower':
        labels -= 20
    labels[labels//10==1] %= 10
    labels[labels//10==2] = (labels[labels//10==2]%10) + 8
    labels[labels<0] = 0
    
    
    ### OBJ 파일로부터 vertices & mesh 읽어들임
    vertices, org_mesh = gu.read_txt_obj_ls(stl_path, ret_mesh=True, use_tri_mesh=False)
    
    
    '''Sampling #1'''
    label_colors = np.zeros(vertices[:, :3].shape)


    ### 레이블 번호별로 색깔(palet) 지정
    for idx, p in enumerate(palet):
        label_colors[labels==idx] = palet[idx]
    
    
    ### (n, 4) 차원에서 [:, -1:] 차원이 아마 opacity라서 1 넣어줌
    label_colors = np.concatenate([label_colors, np.ones((label_colors.shape[0], 1))], axis=-1)
    
    
    ### vertices, vertex normals, vertex colors 로 mesh 생성
    colored_mesh = pymeshlab.Mesh(vertex_matrix = vertices[:, :3],
                                    v_normals_matrix = vertices[:, 3:6],
                                    v_color_matrix = label_colors)
    
    
    ### MeshSet()에 색칠한 mesh 추가 후 샘플링
    colored_mesh_set = pymeshlab.MeshSet()
    colored_mesh_set.add_mesh(colored_mesh)
    
    colored_mesh_set.generate_simplified_point_cloud(radius=pymeshlab.Percentage(0.3), exactnumflag=True)       ## 0.3의 Poisson disk로 샘플링
    
    
    ### 샘플링된 mesh의 vertices의 색깔을 palet 색깔과 비교하여 레이블링
    sampled_label = np.zeros(colored_mesh_set[1].vertex_color_matrix().shape[0])
    for i, p in enumerate(palet):
        sampled_label[((np.isclose(colored_mesh_set[1].vertex_color_matrix()[:,:3], p)).sum(-1)//3).astype(np.bool_)] = i       ## '=='는 부동소수점 문제때문에 색깔이 같다고 인식 못하기 때문에 np.isclose() 사용
    
    
    ### [vertices(3), vertex normals(3), labels(1)] -> (n, 7)
    labeled_vertices = np.concatenate([colored_mesh_set[1].vertex_matrix(), colored_mesh_set[1].vertex_normal_matrix(), np.expand_dims(sampled_label, axis=-1).astype(np.int64)], axis=1)
    
    ''''''
    
    np.save(os.path.join(save_path, base_name+'.npy'), labeled_vertices)

base_name :  4OPR2WF4_upper
[polyscope] [ERROR] name must not be the empty string
base_name :  4OPR2WF4_lower
[polyscope] [ERROR] name must not be the empty string
base_name :  BIIEY91S_upper
[polyscope] [ERROR] name must not be the empty string
base_name :  BIIEY91S_lower
[polyscope] [ERROR] name must not be the empty string
base_name :  85ARPPFY_upper
[polyscope] [ERROR] name must not be the empty string
base_name :  85ARPPFY_lower


RuntimeError: Unable to cast Python instance to C++ type (compile in debug mode for details)

## Osstem dataset

In [8]:
split_base_name_ls = []
stl_path_ls = []
json_path_map = {}

source_path = "../datasets/osttemorigin_annotated"      ## Annotated 오스템 데이터의 데이터 경로

case_list = sorted(os.listdir(source_path))
# case_list = ['Case_05', 'Case_33']

for i, case in enumerate(case_list):
    tgt_path = os.path.join(source_path, 'Case_%02d' % (i+1), 'STL')
    
    
    if os.path.exists(os.path.join(tgt_path, 'Case_%02d_lower.obj' % (i+1))):
        stl_path_ls.append(os.path.join(tgt_path, 'Case_%02d_lower.obj' % (i+1)))
    if os.path.exists(os.path.join(tgt_path, 'Case_%02d_upper.obj' % (i+1))):
        stl_path_ls.append(os.path.join(tgt_path, 'Case_%02d_upper.obj' % (i+1)))
    if os.path.exists(os.path.join(tgt_path, 'Case_%02d_lower.txt' % (i+1))):
        json_path_map[case+'_lower'] = os.path.join(tgt_path, 'Case_%02d_lower.txt' % (i+1))
    if os.path.exists(os.path.join(tgt_path, 'Case_%02d_upper.txt' % (i+1))):
        json_path_map[case+'_upper'] = os.path.join(tgt_path, 'Case_%02d_upper.txt' % (i+1))

In [9]:
len(json_path_map)

92

In [16]:
save_path = 'preprocessed_data_simplification_osstem'

for stl_path in stl_path_ls[20:]:
    base_name = os.path.basename(stl_path).split(".")[0]
    print("base_name : ", base_name)
    
    vertices, org_mesh = gu.read_txt_obj_ls(stl_path, ret_mesh=True, use_tri_mesh=True)
    src_pcd = vertices[:, :6]

    
    ### GT Reading
    ### TXT 파일 안에 있는 번호들은, 스캔 모델 안에 존재하는 치아들의 번호를 의미
    ### OBJ 파일 안에 치아 번호 별로 삼각형들이 그룹화 되어있는데, 그 그룹의 순서가 치아 번호 순서대로 들어있지 않기 때문에 TXT 파일 안의 치아 번호 순서도 중요
    with open(json_path_map[base_name], 'r') as txt_file:
        labels = txt_file.readlines()
        if len(labels)==2:
            teeth = list(map(int, labels[-1].split()))
        else:
            teeth = []
    ###
       
        
    ### OBJ Reading
    with open(stl_path, 'r') as obj_file:
        g, g_id = False, -1
        group_dict, group = {}, set()
        
        for line in obj_file:
            if g and line.startswith('f'):
                group.update(set(map(int, set(line[2:].replace('//', ' ').split()))))
            elif g and line.startswith('#'):
                g = False
                group_dict[teeth[g_id]] = sorted(list(group))
                group = set()
            elif line.startswith('g') and line[9]!='0':
                g = True
                g_id+=1
        if teeth != []:
            group_dict[teeth[g_id]] = sorted(list(group))
            del(group)
        
    gt_labels = np.zeros(len(src_pcd))
    for d in group_dict.keys():
        gt_labels[np.array(group_dict[d])-1] = int(d)

    ### 치아 번호들을 0(잇몸), 1~8(#11~#18, #31~#38), 9~16(#21~#28, #41~48) 로 변경해줌
    if 'lower' in stl_path:
        gt_labels -= 20
    gt_labels[gt_labels//10==1] %= 10
    gt_labels[gt_labels//10==2] = (gt_labels[gt_labels//10==2]%10) + 8
    gt_labels[gt_labels<0] = 0
    ###
    
    
    # '''Sampling #1'''
    label_colors = np.zeros(vertices[:, :3].shape)

    for idx, p in enumerate(palet):
        label_colors[gt_labels==idx] = palet[idx]
        
    label_colors = np.concatenate([label_colors, np.ones((label_colors.shape[0], 1))], axis=-1)
    
    colored_mesh = pymeshlab.Mesh(vertex_matrix = vertices[:, :3],
                                    v_normals_matrix = vertices[:, 3:6],
                                    v_color_matrix = label_colors)
    
    colored_mesh_set = pymeshlab.MeshSet()
    colored_mesh_set.add_mesh(colored_mesh)
    
    colored_mesh_set.generate_simplified_point_cloud(radius=pymeshlab.Percentage(0.3), exactnumflag=True)
    
    sampled_label = np.zeros(colored_mesh_set[1].vertex_color_matrix().shape[0])
    for i, p in enumerate(palet):
        sampled_label[((np.isclose(colored_mesh_set[1].vertex_color_matrix()[:,:3], p)).sum(-1)//3).astype(np.bool_)] = i
    
    labeled_vertices = np.concatenate([colored_mesh_set[1].vertex_matrix(), colored_mesh_set[1].vertex_normal_matrix(), np.expand_dims(sampled_label, axis=-1).astype(np.int64)], axis=1)
    
    ''''''
    
    np.save(os.path.join(save_path, base_name+'.npy'), labeled_vertices)

base_name :  Case_11_lower
(48275,)

base_name :  Case_11_upper
(167537,)

base_name :  Case_12_lower


KeyboardInterrupt: 